# Data loader and package loader

In [ ]:
import torch
from tqdm.auto import tqdm

class GPT_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.example = []

        for each in dataset:
            self.example.append({k: torch.tensor(v) for k, v in each.items()})

    def __getitem__(self, idx):
        return self.example[idx]

    def __len__(self):
        return len(self.example)

In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 5.8 MB 11.4 MB/s 
     |████████████████████████████████| 7.6 MB 69.4 MB/s 
     |████████████████████████████████| 182 kB 67.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel, PreTrainedTokenizerFast, DataCollatorWithPadding
from torch.utils.data import DataLoader

# GPT test

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token='<sep>')
tokenizer.padding_side = 'left'

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.to('cuda')

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/level2_RE/train.csv',index_col=0)

In [ ]:
# 문장을 생성하는 함수
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True)).unsqueeze(0).to('cuda')

    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=120, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=10,
        eos_token_id=tokenizer.encode('</s>')[0],
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [ ]:
# 예시 문장
print(train_df.sentence.iloc[0])
print(train_df.sentence.iloc[1])
print(train_df.sentence.iloc[2])
print(train_df.sentence.iloc[3])
print(train_df.sentence.iloc[4])

〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.
: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니.


In [ ]:
print(train_df.subject_entity.iloc[0], train_df.object_entity.iloc[0])
print(train_df.subject_entity.iloc[1], train_df.object_entity.iloc[1])
print(train_df.subject_entity.iloc[2], train_df.object_entity.iloc[2])
print(train_df.subject_entity.iloc[3], train_df.object_entity.iloc[3])
print(train_df.subject_entity.iloc[4], train_df.object_entity.iloc[4])

{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'} {'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}
{'word': '광주FC', 'start_idx': 21, 'end_idx': 24, 'type': 'ORG'} {'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx': 41, 'type': 'ORG'}
{'word': '아성다이소', 'start_idx': 13, 'end_idx': 17, 'type': 'ORG'} {'word': '박정부', 'start_idx': 22, 'end_idx': 24, 'type': 'PER'}
{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_idx': 30, 'type': 'ORG'} {'word': '1967', 'start_idx': 0, 'end_idx': 3, 'type': 'DAT'}
{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_idx': 22, 'type': 'ORG'} {'word': 'NATO', 'start_idx': 25, 'end_idx': 28, 'type': 'ORG'}


In [ ]:
print(get_gpt_output('</s>〈Something〉는 조지 해리슨이 쓰고 비틀즈'))
print(get_gpt_output('</s>K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹'))
print(get_gpt_output('</s>균일가 생활용품점 (주)아성다이소(대표 박정부'))
print(get_gpt_output('</s>1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠'))
print(get_gpt_output('</s>: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO'))

# pre-train 를 진행하기 이전에 ko/gpt2 만으로 문장을 생성한 결과.
# 자연스러운 문장이 생성되기는 하지만 하나의 문장이 종료가 되고 다시 그닥 연관이 없는 문장이 계속 생성되는 것을 확인 할 수 있다.

〈Something〉는 조지 해리슨이 쓰고 비틀즈 멤버인 제이미 로드가 불렀으며 ‘South of Finals’ 로고와 함께 이 곡으로 로고송이 바뀌었다.
이 곡이 부른 ‘Don’t Stop’은 Sometimate arts의 2곡 모음곡으로 수록되었는데, 여기에 가사를 쓰고 있는 제이미한테 가사를 쓰게 되는 순간부터 다시 한 번 로고가 쓰인 ‘Your Home, Somewhere’가 시작되었다.
이후 이 곡을 녹음한 Jack Punk에서 〈Hope for Day〉의 〈Leaves Y
K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹(KFA) 광주유나이티드에 1부리그 승격을 신청했다.
광주FC와 KEB하나은행은 최근 프로축구 본부의 지원과 연고지 변경을 위해 이사회를 열었으나 KBO 이사회가 오는 29일로 예정돼 있어 최종 결정을 내리지 못하고 있다.
이 때문에 KAFA 이사회는 이날 임시주주총회를 소집하는 등 본격적인 절차의 시작을 알리는데 주력했다.
KB국민은행은 KFL 출범 이후 줄곧 KOVO를 광주에 1위로 올려 놓았으나 올해부터 광주와
균일가 생활용품점 (주)아성다이소(대표 박정부)의 '수퍼카'와 '쿠팡'(대표 손영기)의 친환경 캠페인 '티맵(TIMA)'이 그 주인공이다.
지난해 10월 오픈한 '슈퍼 카'는 자동차 엔진으로 구동되는 티맵과 일반 카와 같이 일반 모델처럼 시동을 걸 수 있는 하이브리드 시스템을 적용했다.
티켓 가격이 비싸다는 지적을 받는 등 사회적 관심이 높아지자 티켓 가격을 낮춘 것도 특징이다.
티몬에서는 티몬 로켓배송 상품 구매 시 최대 30% 할인된 가격에 티비쇼핑
1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에 지명됐다.
부상으로 한 시즌을 쉬고 퇴장한 요나카는 2년째 2군에 머문다. 서울시가 '서울장터' 조성 사업에 뛰어들었다.
서울시는 28일 서울장터에 참여하면서 한류 체험과 외국인들을 위한 무료입장권을 추가로 제공한다.
시는 이날부터 오는 7월까지 서울 장터 홈페이지를 통해 서울 대표 명물

# GPT-2 Pretrain 1

label 정보와 sentence 정보를 같이 pre-train 시킴으로써 훈련 데이터들과 유사한 문장이 생성됨과 하게 동시에 label에 대한 정보도 생성된 문장에 담음.
prompt로 label 정보도 포함할시 label과 일관성이 있는 데이터를 생성되게함

In [ ]:
def tokenized_dataset(dataset, tokenizer):

    data = []
    for idx, item in tqdm(dataset.iterrows(), desc="tokenizing", total=len(dataset)):

        # '</s>label<sep>sentence</s>' 형태로 pre-train 시행

        tmp_sentence = '</s>'
        tmp_sentence += item['label']
        tmp_sentence += '<sep>'
        tmp_sentence += item['sentence']
        tmp_sentence += '</s>'

        output = tokenizer(tmp_sentence, padding=True, truncation=True, add_special_tokens=True)
        data.append(output)

    return data

def load_dataset(tokenizer, data_path):
    dataset = pd.read_csv(data_path, index_col=0)
    label = list(dataset.label.unique())
    tokenizer.add_tokens(label)

    tokenized_data = tokenized_dataset(dataset, tokenizer)

    GPT_dataset = GPT_Dataset(tokenized_data)
    return GPT_dataset

In [ ]:
dataset = load_dataset(tokenizer,'/content/drive/MyDrive/level2_RE/train.csv')

tokenizing:   0%|          | 0/25976 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
dataset.example[0]

{'input_ids': tensor([    1, 51201, 51200,  9151,   425, 11308, 30190, 10929, 11953, 12889,
         23821, 25478, 13296, 21902, 38937, 15451, 22973, 11072,   407,   440,
           440,   443,   463, 35413, 12293, 17115, 16637, 10121,  9016,     1]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1])}

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from transformers import DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)

In [ ]:
data_collator = DataCollatorForLanguageModeling(    # GPT는 생성모델이기 때문에 [MASK] 가 필요 없습니다 :-)
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
len(tokenizer)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=7,
    per_device_train_batch_size=16, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()
trainer.save_model('/content/drive/MyDrive/level2_RE/GPT2')

In [ ]:
# 훈련된 모델을 불러오기
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2')
model.to('cuda')

### 결과 확인

In [ ]:
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True)).unsqueeze(0).to('cuda')

    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=120, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1,
        eos_token_id=tokenizer.encode('</s>')[0],
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [ ]:
# 예시 원문
print(train_df.sentence.iloc[2], train_df.label.iloc[2])

균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다. org:top_members/employees


In [ ]:
print(get_gpt_output('</s>〈Something〉는 조지 해리슨이 쓰고 비틀즈'))
print(get_gpt_output('</s>K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹'))
print(get_gpt_output('</s>균일가 생활용품점 (주)아성다이소(대표 박정부'))
print(get_gpt_output('</s>1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠'))
print(get_gpt_output('</s>: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO'))

〈Something〉는 조지 해리슨이 쓰고 비틀즈에 발표한 1964년 영국의 록 밴드 비틀즈의 일곱 번째 정규 앨범이다.
K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 성적 75점으로 FA컵 3관왕의 영광을 안았다.
균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 확산에 따른 대구 지역 사회의 대응과 관련해 대구광역시와 경상북도, 대구시의 공동 대응을 위해 아동복지시설 등 관련시설에 대한 27일부터 다음달 15일까지 일시적 휴원을 시행한다고 밝혔다.
1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에 지명됐다.
: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO)의 안전보장과 유럽 차원의 안전 확보가 시급하다.


In [ ]:
example_idx = 30
print(train_df.sentence.iloc[example_idx])
print("subject :", eval(train_df.subject_entity.iloc[example_idx])['word'])
print("object :", eval(train_df.object_entity.iloc[example_idx])['word'])
print("label :", train_df.label.iloc[example_idx])

광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리가 호남의 판소리를 전 세계에 알린다.
subject : 박애리
object : 국악인
label : per:title


In [ ]:
print(get_gpt_output('</s>per:colleagues<sep>1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희'))
print(get_gpt_output('</s>per:colleagues<sep>1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희'))
print(get_gpt_output('</s>per:colleagues<sep>1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희'))
print(get_gpt_output('</s>per:colleagues<sep>1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희'))
print(get_gpt_output('</s>per:colleagues<sep>1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희'))

per:colleagues 1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희를 38선 후보로 추대하였다.
per:colleagues 1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희에게 48 대 56으로 패배했으나 그 해 부통령 후보에 올랐고 1973년 12월 10일 통일민주당을 탈당하여 노태우에게 대통령 예비 후보되었다.
per:colleagues 1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희에게 패한 직후 당선을 위해 아내 김현옥과 함께 신군부 세력으로부터 지지를 얻어내려 했다.
per:colleagues 1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희를 지지했으며, 대선 전날인 1974년 5월 27일 평화방송 《열린세상 오늘!》에 다음과 같은 내용의 사과문을 발표했다.
per:colleagues 1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희에 이어 2위로 신민당 정권을 꺾고 대통령에 당선되었지만 2년 후 1974년 1월에 발생한 IMF 외환위기 사태를 직접 언급하면서 유신당을 비판한 것을 문제로 삼으며 재협상을 요구하였고 이에 김종필의 사퇴를 요구하는 목소리가 높았으나 계속된 협상에도 자신이 유신당 정권의 수반이 되어 1971년 4월 28일 대선에서 패배하여 원점에서 복귀하였다.


In [ ]:
print(get_gpt_output('</s>per:title<sep>광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리'))
print(get_gpt_output('</s>per:title<sep>광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리'))
print(get_gpt_output('</s>per:title<sep>광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리'))
print(get_gpt_output('</s>per:title<sep>광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리'))
print(get_gpt_output('</s>per:title<sep>광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리'))

per:title 광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리씨가 국악팬에게 감사의 글을 남겼다.
per:title 광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리 씨가 전라남도와 목포시에 공연 홍보를 위한 특별 공연을 선보여 호응을 얻고 있다.
per:title 광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리 씨가 국립아시아문화전당 공연장과 팬 사인회를 통해 광주 모습을 처음으로 공개했다.
per:title 광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리 씨가 전남지역 대표 국악공연으로 새로운 정서와 재미를 더한다.
per:title 광주국악상설공연이 세계수영선수권대회 기간동안 연이어 매진을 기록하며 광주 대표 문화관광 콘텐츠로 큰 호응을 받고 있는 가운데 국악인 박애리 씨가 새로운 주역으로 출연해 호남 대표 국악지인 대중음악계와 소통하며 흥겨운 무대를 선사할 예정이다.


In [ ]:
get_gpt_output('</s>org:top_members/employees<sep>균일가 생활용품점 (주)아성다이소')

'org:top_members/employees 균일가 생활용품점 (주)아성다이소 잡화부문 채학봉 담당이 ‘코로나19’ 극복을 위해 성금 2억 원을 기부했다.'

In [ ]:
get_gpt_output('</s>org:top_members/employees<sep>균일가 생활용품점 (주)아성다이소')

'org:top_members/employees 균일가 생활용품점 (주)아성다이소 고호석(대표 박정부)은 코로나19 극복을 위해 2월 27일부터 28일까지 3일간 한시적으로 ‘강서구 우수 중소기업 상품전’을 개최한다.'

In [ ]:
# 원문과 비슷한 의미의 문장들이 생성됨
# 이전 처럼 하나의 문장이 끝나고 별 상관없는 문장이 계속 생성되는것이 아닌 하나의 일관성 있는 문장만 생성이됨

# GPT-2 Pretrain 2

이번에는 label뿐만 아니라 subject_entity와 object entity를 같이 학습 시킴으로써
문장 생성과 동시에 entity도 자동으로 생성되게 함

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token='<sep>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.add_special_tokens({'additional_special_tokens':['<subj>','<obj>']})

2

In [ ]:
def tokenized_dataset2(dataset, tokenizer):

    data = []
    for idx, item in tqdm(dataset.iterrows(), desc="tokenizing", total=len(dataset)):

        # </s>label<sep>sentence<subj>e_subj<subj><obj>e_obj<obj></s> 와 같은 형태로 학습
        tmp_sentence = '</s>'
        tmp_sentence += item['label']
        tmp_sentence += '<sep>'
        tmp_sentence += item['sentence']
        tmp_sentence += '<subj>'+ eval(item['subject_entity'])["word"] + '<subj>'
        tmp_sentence += '<obj>' + eval(item['object_entity'])["word"] + '<obj>'
        tmp_sentence += '</s>'

        output = tokenizer(tmp_sentence, padding=True, truncation=True, add_special_tokens=True)
        data.append(output)

    return data

In [ ]:
def load_dataset2(tokenizer, data_path):
    dataset = pd.read_csv(data_path, index_col=0)
    label = list(dataset.label.unique())
    tokenizer.add_tokens(label)

    tokenized_data = tokenized_dataset2(dataset, tokenizer)

    GPT_dataset = GPT_Dataset(tokenized_data)
    return GPT_dataset

In [ ]:
dataset = load_dataset2(tokenizer,'/content/drive/MyDrive/level2_RE/train.csv')

tokenizing:   0%|          | 0/25976 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)

In [ ]:
data_collator = DataCollatorForLanguageModeling(    # GPT는 생성모델이기 때문에 [MASK] 가 필요 없습니다 :-)
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
# 51231 => <subj>, 51232 => <obj> special token이 포함된 형태
dataset.example[0]['input_ids']

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()
trainer.save_model('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token')

### 결과 확인

In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token')
model.to(device)

In [ ]:
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True)).unsqueeze(0).to(device)

    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=120, 
        top_k=50, 
        top_p=0.80, 
        num_return_sequences=1,
        eos_token_id=tokenizer.encode('</s>')[0],
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [ ]:
print(train_df.sentence.iloc[2])
print("subject :", eval(train_df.subject_entity.iloc[2])['word'])
print("object :", eval(train_df.object_entity.iloc[2])['word'])
print("label :", train_df.label.iloc[2])

균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
subject : 아성다이소
object : 박정부
label : org:top_members/employees


In [ ]:
get_gpt_output('</s>org:top_members/employees<sep>균일가 생활용품점 (주)아성다이소')

'org:top_members/employees 균일가 생활용품점 (주)아성다이소공업(이하 다이소)이 코로나19 바이러스 감염증(이른바 코호트) 극복을 위해 후원 물품들을 전달했다고 13일 밝혔다. 다이소스 김지수'

In [ ]:
get_gpt_output('</s>org:top_members/employees<sep>균일가 생활용품점 (주)아성다이소')

'org:top_members/employees 균일가 생활용품점 (주)아성다이소 성인용품점(대표 박정부)이 코로나19 감염증 확산으로 어려움을 겪고 있는 대구광역시에 마스크 1만개를 전달했다고 2일 밝혔다. 다이소 박정부와정부'

In [ ]:
get_gpt_output('</s>org:top_members/employees<sep>균일가 생활용품점 (주)아성다이소')

'org:top_members/employees 균일가 생활용품점 (주)아성다이소녀 대표가 코로나19 여파로 영업에 어려움을 겪고 있는 소상공인들을 위해 상생 지원을 나섰다. 아성다이스 김성다이소스'

In [ ]:
print(train_df.sentence.iloc[4])
print("subject :", eval(train_df.subject_entity.iloc[4])['word'])
print("object :", eval(train_df.object_entity.iloc[4])['word'])
print("label :", train_df.label.iloc[4])

: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니.
subject : 북대서양 조약 기구
object : NATO
label : org:alternate_names


In [ ]:
get_gpt_output('</s>org:alternate_names<sep>: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO)')

'org:alternate_names : 유엔, 유럽 의회, 북대서양 조약 기구 (NATO) 및 국제 연합, 그리고 국제 통화 기금(IMF)에 의한 세계 경제 원조 및 보호에 힘입은 국가들끼리 협력 할 수 있는 방안을 모색하고 모색한다. 북대서양 조약 NATO'

In [ ]:
get_gpt_output('</s>org:alternate_names<sep>: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO)')

'org:alternate_names : 유엔, 유럽 의회, 북대서양 조약 기구 (NATO) 및 국제 적십자 기구들 (ICRC), 유엔 안전 보장 이사회 (유엔, 국제 연합), 유럽 이사회 상임 이사국 (2019년, G20), 글로벌 기업 (2020년)을 비롯한 많은 국가 및 기관과 단체가 참여하며, 이 중 유엔은 국제 의무에 규정되어 있다. 북대서양 조약 NATO'

In [ ]:
get_gpt_output('</s>org:alternate_names<sep>: 유엔, 유럽 의회, 북대서양 조약 기구 (NATO)')

'org:alternate_names : 유엔, 유럽 의회, 북대서양 조약 기구 (NATO) 및 국제 연합 (GSOMIA), 유럽 안보 협력 기구, 세계 보건 기구 (1963년 설립) 및 세계 대전 기간 동안 미군 점령 지역 시 전투에 참여한 것으로 공식적으로 인정받은 전쟁 포로, 영국/독일군 및 기타 연합군의 전투 포로를 받았다. 북대유럽 조약 기구 NATO'

In [ ]:
# entity를 자동으로 찾아주도록 학습을 하였으나 생성된 문장의 자연스러움을 위해 top-k의 option들을 사용하다보니 생성된 entity들도 부정확 할때가 많음

# GPT-2 Pretrain 3

이번에는 label과 subject, object를 동시에 prompt를 주면 어떻게 될지 궁금해서 한번 학습을 해봄

In [ ]:
def tokenized_dataset3(dataset, tokenizer):

    data = []
    for idx, item in tqdm(dataset.iterrows(), desc="tokenizing", total=len(dataset)):

        tmp_sentence = '</s>'
        tmp_sentence += item['label']
        tmp_sentence += '<sep>'
        tmp_sentence += '<subj>'+ eval(item['subject_entity'])["word"] + '<subj>'
        tmp_sentence += '<obj>' + eval(item['object_entity'])["word"] + '<obj>'
        tmp_sentence += '<sep>'
        tmp_sentence += item['sentence']
        tmp_sentence += '</s>'

        output = tokenizer(tmp_sentence, padding=True, truncation=True, add_special_tokens=True)
        data.append(output)

    return data

In [ ]:
def load_dataset3(tokenizer, data_path):
    dataset = pd.read_csv(data_path, index_col=0)
    label = list(dataset.label.unique())
    tokenizer.add_tokens(label)

    tokenized_data = tokenized_dataset3(dataset, tokenizer)

    GPT_dataset = GPT_Dataset(tokenized_data)
    return GPT_dataset

In [ ]:
dataset = load_dataset3(tokenizer,'/content/drive/MyDrive/level2_RE/train.csv')

tokenizing:   0%|          | 0/25976 [00:00<?, ?it/s]

In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)

In [ ]:
data_collator = DataCollatorForLanguageModeling(    # GPT는 생성모델이기 때문에 [MASK] 가 필요 없습니다 :-)
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()
trainer.save_model('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token_first')

### 결과 확인

In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token_first')
model.to(device)

In [ ]:
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True)).unsqueeze(0).to(device)

    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=120, 
        top_k=50, 
        top_p=0.80, 
        num_return_sequences=1,
        eos_token_id=tokenizer.encode('</s>')[0],
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [ ]:
print(train_df.sentence.iloc[2])
print("subject :", eval(train_df.subject_entity.iloc[2])['word'])
print("object :", eval(train_df.object_entity.iloc[2])['word'])
print("label :", train_df.label.iloc[2])

균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
subject : 아성다이소
object : 박정부
label : org:top_members/employees


In [ ]:
get_gpt_output('</s>org:top_members/employees<sep><subj>아성다이소<subj><obj>박정부<obj>')

'org:top_members/employees 아성다이소 박정부 균일가 생활용품점 (주)아성다이(대표 박정부와 문효옥)가 코로나19 피해 복구에 써달라며 성금 1천만원을 기탁했다.'

In [ ]:
get_gpt_output('</s>org:top_members/employees<sep><subj>아성다이소<subj><obj>박정부<obj>')

'org:top_members/employees 아성다이소 박정부 균일가 생활용품점 (주)아성다이(대표 박정부의)는 코로나19 확산으로 어려움을 겪고 있는 소상공인을 돕기 위해 ‘착한 임대료 운동’을 전개한다고 17일 밝혔다.'

In [ ]:
get_gpt_output('</s>org:top_members/employees<sep><subj>아성다이소<subj><obj>박정부<obj>')

'org:top_members/employees 아성다이소 박정부 균일가 생활용품점 (주)아성다이(대표 박정부의)가 코로나19의 확산으로 어려움을 겪고 있는 소상공인들의 애로사항을 청취하고 대책을 논의하는 자리를 가졌다.'

In [ ]:
print(train_df.sentence.iloc[5])
print("subject :", eval(train_df.subject_entity.iloc[5])['word'])
print("object :", eval(train_df.object_entity.iloc[5])['word'])
print("label :", train_df.label.iloc[5])

중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 것을 결정해, 남아있는 중화민국군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다.
subject : 중화민국
object : 타이베이
label : org:place_of_headquarters


In [ ]:
get_gpt_output('</s>org:place_of_headquarters<sep><subj>중화민국<subj><obj>타이베이<obj>')

'org:place_of_headquarters 중화민국 타이베이 이러한 이유로 중화민국은 타이베이로 가는 항공편을 감편 운항하고 있다.'

In [ ]:
get_gpt_output('</s>org:place_of_headquarters<sep><subj>중화민국<subj><obj>타이베이<obj>')

'org:place_of_headquarters 중화민국 타이베이 저우융캉(주영강, 1965년 10월 16일 ~)은 중화민국의 야구 선수이자 전 중화인민공화국 축구 국가대표팀 감독이다.'

In [ ]:
get_gpt_output('</s>org:place_of_headquarters<sep><subj>중화민국<subj><obj>타이베이<obj>')

'org:place_of_headquarters 중화민국 타이베이 중화민국은 타이베이로 수도를 이전한 이후 지금까지 타이완 전지역, 신좡 분리주의 진저우를 타이만의 도시로 삼고 있다.'

어느 정도 label, subj, obj prompt에 기반한 문장이 생성되는것을 확인하였으나 항상 정확한 문장이 생성되는것은 아님. 
generate의 다양한 옵션이 원인인것으로 생각된다.

# Gpt-2 data generate -1

여러가지 prompt를 주는 방법을 시도해보았으나 subj_entity와 obj_entity간의 관계가 부정확한 문장이 생성되는 빈도가 더 높아서 그냥 label만 prompt로 주어서 데이터 증강을 해보기로함

In [ ]:
# label 정보와 entity 정보를 그대로 보존하기 위해서
# 라벨과 하나의 row에 대한 entity 두개 중 idx의 최대값까지만 문장을 인덱싱하여 prompt를 줌
# prompt : </s> label <sep> sentence[:max_idx]
# 예시
# 문장 : 균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
# subject : 아성다이소
# object : 박정부
# label : org:top_members/employees
# prompt : </s>org:top_members/employees<sep>균일가 생활용품점 (주)아성다이소(대표 박정부

# 또는 아래와 같은 형태로 증강
# 문장 : 균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
# subject : 아성다이소
# object : 박정부
# label : org:top_members/employees
# prompt : </s>org:top_members/employees<sep>(주)아성다이소(대표 박정부

In [ ]:
device = 'cuda'

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/level2_RE/train.csv',index_col=0)
train_df = train_df.loc[train_df.label == 'org:p'].reset_index(drop=True)

In [ ]:
class GPT_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.example = []

        for each in dataset:
            self.example.append({k: torch.tensor(v) for k, v in each.items()})

    def __getitem__(self, idx):
        return self.example[idx]

    def __len__(self):
        return len(self.example)

def tokenized_dataset(dataset, tokenizer):

    data = []
    for idx, item in tqdm(dataset.iterrows(), desc="tokenizing", total=len(dataset)):

        subj_start_idx = eval(item['subject_entity'])['start_idx']
        subj_end_idx = eval(item['subject_entity'])['end_idx']
        obj_start_idx = eval(item['object_entity'])['start_idx']
        obj_end_idx = eval(item['object_entity'])['end_idx']

        min_idx = min([subj_start_idx, subj_end_idx, obj_start_idx, obj_end_idx])
        max_idx = max([subj_start_idx, subj_end_idx, obj_start_idx, obj_end_idx])

        tmp_sentence = '</s>'
        tmp_sentence += item['label']
        tmp_sentence += '<sep>'
        tmp_sentence += item['sentence'][:max_idx+1]
        tmp_sentence += '</s>'
        output = tokenizer(tmp_sentence, padding=True, truncation=True, add_special_tokens=True)
        data.append(output)

    return data

def load_dataset(tokenizer, dataset):
    tokenized_data = tokenized_dataset(dataset, tokenizer)
    GPT_dataset = GPT_Dataset(tokenized_data)
    return GPT_dataset

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token='<sep>')

tokenizer.padding_side = 'left'

label = list(train_df.label.unique())
tokenizer.add_tokens(label)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2')
#model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token')
#model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token_first')
model.to(device)
model.eval()

In [ ]:
dataset = load_dataset(tokenizer,train_df)
data_collator = DataCollatorWithPadding(tokenizer)
data_tobe_generated = DataLoader(dataset, batch_size=64, collate_fn=data_collator)
len(data_tobe_generated)

tokenizing:   0%|          | 0/7640 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


120

In [ ]:
# import gc
# torch.cuda.empty_cache()
# gc.collect()

In [ ]:
output = []
for each in tqdm(data_tobe_generated):
    
    sample_outputs = model.generate(
        each['input_ids'].to(device),
        do_sample=True, 
        max_length=len(each['input_ids'][0]) + 50, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1,
        eos_token_id=tokenizer.encode('</s>')[0],
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    result = tokenizer.batch_decode(sample_outputs.tolist(), skip_special_tokens=True)
    decoded_sentence = list(map(lambda x : ' '.join(x.split(' ')[1:]),result))

    output.extend(decoded_sentence)


  0%|          | 0/120 [00:00<?, ?it/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
idx = 300
print(train_df.sentence.iloc[idx])
print(train_df.subject_entity.iloc[idx], train_df.object_entity.iloc[idx])
print(output[idx])

그 당시 NHL 캘거리 플레임스에 4순위로도 지명되었고, 미네소타 대학에서는 야구와 아이스하키 선수를 겸업하는 조건으로 장학금을 제의하기도 했다.
{'word': 'NHL', 'start_idx': 5, 'end_idx': 7, 'type': 'ORG'} {'word': '아이스하키', 'start_idx': 47, 'end_idx': 51, 'type': 'POH'}
그 당시 NHL 캘거리 플레임스에 4순위로도 지명되었고, 미네소타 대학에서는 야구와 아이스하키 외야수로서의 길을 택하였다.


In [ ]:
train_df.sentence = output

In [ ]:
len(output)

7640

In [ ]:
train_df.to_csv('no_relation.csv')

# GPT-2 data generate-2

In [ ]:
def find_nth_overlapping(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+1)
        n -= 1
    return start

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/level2_RE/train.csv',index_col=0)

In [ ]:
to_generated_df = train_df.loc[train_df.label == 'per:origin'].reset_index(drop=True)

In [ ]:
class GPT_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.example = []

        for each in dataset:
            self.example.append({k: torch.tensor(v) for k, v in each.items()})

    def __getitem__(self, idx):
        return self.example[idx]

    def __len__(self):
        return len(self.example)

def tokenized_dataset(dataset, tokenizer):

    data = []
    for idx, item in tqdm(dataset.iterrows(), desc="tokenizing", total=len(dataset)):

        subj_start_idx = eval(item['subject_entity'])['start_idx']
        subj_end_idx = eval(item['subject_entity'])['end_idx']
        obj_start_idx = eval(item['object_entity'])['start_idx']
        obj_end_idx = eval(item['object_entity'])['end_idx']

        min_idx = min([subj_start_idx, obj_start_idx])
        max_idx = max([subj_end_idx, obj_end_idx])

        k_idx = find_nth_overlapping(item['sentence'][max_idx:],' ',1)

        tmp_sentence = '</s>'
        tmp_sentence += item['label']
        tmp_sentence += '<sep>'
        tmp_sentence += item['sentence'][:max_idx+k_idx]
        tmp_sentence += '</s>'

        output = tokenizer(tmp_sentence, padding=True, truncation=True, add_special_tokens=True)
        data.append(output)

    return data

def load_dataset(tokenizer, dataset):
    tokenized_data = tokenized_dataset(dataset, tokenizer)
    GPT_dataset = GPT_Dataset(tokenized_data)
    return GPT_dataset

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token='<sep>')

tokenizer.padding_side = 'left'

label = list(train_df.label.unique())
tokenizer.add_tokens(label)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


30

In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2')
#model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token')
#model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/level2_RE/GPT2_with_label_special_token_first')
model.to(device)
model.eval()

In [ ]:
dataset = load_dataset(tokenizer,to_generated_df)
data_collator = DataCollatorWithPadding(tokenizer)
data_tobe_generated = DataLoader(dataset, batch_size=128, collate_fn=data_collator)
len(data_tobe_generated)

tokenizing:   0%|          | 0/996 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


8

In [ ]:
output = []

for each in tqdm(data_tobe_generated):
    
    sample_outputs = model.generate(
        each['input_ids'].to(device),
        do_sample=True, 
        max_length=len(each['input_ids'][0]) + 100, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1,
        eos_token_id=tokenizer.encode('</s>')[0],
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    result = tokenizer.batch_decode(sample_outputs.tolist(), skip_special_tokens=True)
    decoded_sentence = list(map(lambda x : ' '.join(x.split(' ')[1:]),result))

    output.extend(decoded_sentence)

  0%|          | 0/8 [00:00<?, ?it/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
idx = 10
print(to_generated_df.sentence.iloc[idx])
print(to_generated_df.subject_entity.iloc[idx], to_generated_df.object_entity.iloc[idx])
print(output[idx])

나중에는 도조 히데키의 일본 제국 정부와도 손을 잡아 추축국을 만들었다.
{'word': '도조 히데키', 'start_idx': 5, 'end_idx': 10, 'type': 'PER'} {'word': '일본 제국', 'start_idx': 13, 'end_idx': 17, 'type': 'ORG'}
나중에는 도조 히데키의 일본 제국 히데키는 메이지 유신 말기에 쇼군인 도쿠가와 이에모치의 양자가 된다.


In [ ]:
to_generated_df.sentence = output

In [ ]:
to_generated_df

,sentence,subject_entity,object_entity,label,source
0,하비에르 파스토레는 아르헨티나 아르헨티나인 어머니 프란치스카 프루소 사이에서 태어났다.,"{'word': '하비에르 파스토레', 'start_idx': 0, 'end_idx...","{'word': '아르헨티나', 'start_idx': 11, 'end_idx': ...",per:origin,wikipedia
1,"1900년, 의화단의 난이 일어나 청나라 조정이 열강에 선전 포고를 했을 때에, 이...","{'word': '유곤일', 'start_idx': 55, 'end_idx': 57...","{'word': '청나라', 'start_idx': 19, 'end_idx': 21...",per:origin,wikipedia
2,민 리(1977년 6월 27일 ~)는 베트남계 캐나다인 배우이다.,"{'word': '민 리', 'start_idx': 0, 'end_idx': 2, ...","{'word': '캐나다', 'start_idx': 26, 'end_idx': 28...",per:origin,wikipedia
3,리사 디 배나는 중국에서 개최된 2007년 FIFA 여자 월드컵에서 노르웨이와의 조...,"{'word': '리사 디 배나', 'start_idx': 0, 'end_idx':...","{'word': '오스트레일리아', 'start_idx': 54, 'end_idx'...",per:origin,wikipedia
4,"김영삼 대통령 퇴임 후 미국 아이오와 주립대학 객원연구원으로 있다가, 1999년 귀...","{'word': '노무현', 'start_idx': 95, 'end_idx': 97...","{'word': '대한민국', 'start_idx': 51, 'end_idx': 5...",per:origin,wikipedia
...,...,...,...,...,...
991,키라 코르피는 2005년 핀란드 스웨덴어인 타란토어 단어 garanto 에서 유래한...,"{'word': '키라 코르피', 'start_idx': 0, 'end_idx': ...","{'word': '핀란드', 'start_idx': 14, 'end_idx': 16...",per:origin,wikipedia
992,"2016년 20세기 멕시코를 대표하는 부부 프리다 칼로와 카를로스 슬림, 카를로스 ...","{'word': '프리다 칼로', 'start_idx': 24, 'end_idx':...","{'word': '멕시코', 'start_idx': 11, 'end_idx': 13...",per:origin,wikipedia
993,이들은 1882년 6월 6일자로 청나라 마건충(馬建忠)·정여창 아편 흡연자 건륭제의...,"{'word': '마건충', 'start_idx': 22, 'end_idx': 24...","{'word': '청나라', 'start_idx': 18, 'end_idx': 20...",per:origin,wikipedia
994,"그 어떤 수학자들보다 많은 책을 집필한 것으로 유명한 레온하르트 오일러, 오일러에게...","{'word': '레온하르트 오일러', 'start_idx': 30, 'end_id...","{'word': '스위스', 'start_idx': 67, 'end_idx': 69...",per:origin,wikipedia


In [ ]:
to_generated_df.to_csv('per:origin.csv')